# Alignment and annotation: 
Human glioblastoma tumors single nucleus transcripts were retrieved from Gene Expression Omnibus (GEO), with identifier GSE174554. Organoid and human glioblastoma transcripts were aligned and annotated using the NF-Core scrnaseq pipeline and the Cell Ranger mkref and mkgtf functions using the GRCh38.p14 reference genome sequence (fasta) and gene annotations (gtf). Raw sequencing reads underwent quality control using FastQC. Gene-level (counts) were quantified using Cell Ranger count. The resulting count matrix was converted into an H5AD file format (Scanpy compatible), and quality control metrics were collected by MultiQC.

The following steps take in these .h5 files.

### Initial Setup
Initial setup for our required packages and setting how verbose our errors should be.
During install, you may need to use ```pip``` rather thna ```pip3```, depends on how python is installed on your system (and what verions you have).

Also, we are suppressing some terminal output using ```grep``` (so the output looks a little cleaner).

In [ ]:
!pip3 install numpy | grep -v 'already satisfied'
!pip3 install pandas | grep -v 'already satisfied'
!pip3 install seaborn | grep -v 'already satisfied'
!pip3 install scanpy | grep -v 'already satisfied'
!pip3 install leidenalg | grep -v 'Requirement already satisfied'
!pip3 install louvain | grep -v 'Requirement already satisfied'

import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.1 MB/s eta 0:00:0000:0100:010m


/Users/shaunie/Desktop/hu-glioblastoma/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.9.6 anndata==0.10.4 umap==0.5.5 numpy==1.26.3 scipy==1.11.4 pandas==2.1.4 scikit-learn==1.3.2 statsmodels==0.14.1 igraph==0.10.8 louvain==0.8.1 pynndescent==0.5.11


In [ ]:
# Aprrox. 1 sec per read
file_location = ''
data_1914 = '1914/1914_filtered.h5'
data_1914d = '1914d/1914d_filtered.h5'
data_1919 = '1919/1919_filtered.h5'
data_1919d = '1919d/1919d_filtered.h5'

# For remote server
# file_location = '../datasets/'

adata_1914_human = sc.read_10x_h5(
    file_location + data_1914,
    genome='GRCh38')
adata_1914d_human = sc.read_10x_h5(
    file_location + data_1914d,
    genome='GRCh38')
adata_1919_human = sc.read_10x_h5(
    file_location + data_1919,
    genome='GRCh38')
adata_1919d_human = sc.read_10x_h5(
    file_location + data_1919d,
    genome='GRCh38')

adata_1914_rat = sc.read_10x_h5(
    file_location + data_1914,
        genome='mRatBN7.2')
adata_1914d_rat = sc.read_10x_h5(
    file_location + data_1914d,
    genome='mRatBN7.2')
adata_1919_rat = sc.read_10x_h5(
    file_location + data_1919,
    genome='mRatBN7.2')
adata_1919d_rat = sc.read_10x_h5(
    file_location + data_1919d,
    genome='mRatBN7.2')

# View the first 5 rows of the data
adata_1914_human.var.head()
adata_1914_human.obs.head()



### Result:
We see that everything is either prepended by ```GRCh38____``` or ```mRatBN7.2_```. 
Thankfully, it seems that whoever did the previous analysis made sure these prepended labels were always ten charecters by adding underscores, so we can just strip the first ten charecters off all the gene_ids.

The first thing we are going to do is strip the first ten charecters from all the gene_ids. 

Then we are going to ensure that all the gene_ids are unique in the respective datasets.

In [ ]:
# filter the GRCh38__ from the front of names
adata_1914_human.var_names = [name[10:] for name in adata_1914_human.var_names]
adata_1914d_human.var_names = [name[10:] for name in adata_1914d_human.var_names]
adata_1919_human.var_names = [name[10:] for name in adata_1919_human.var_names]
adata_1919d_human.var_names = [name[10:] for name in adata_1919d_human.var_names]
adata_1914_rat.var_names = [name[10:] for name in adata_1914_rat.var_names]
adata_1914d_rat.var_names = [name[10:] for name in adata_1914d_rat.var_names]
adata_1919_rat.var_names = [name[10:] for name in adata_1919_rat.var_names]
adata_1919d_rat.var_names = [name[10:] for name in adata_1919d_rat.var_names]


# make the gene names unique
adata_1914_human.var_names_make_unique()
adata_1914d_human.var_names_make_unique()
adata_1919_human.var_names_make_unique()
adata_1919d_human.var_names_make_unique()
adata_1914_rat.var_names_make_unique()
adata_1914d_rat.var_names_make_unique()
adata_1919_rat.var_names_make_unique()
adata_1919d_rat.var_names_make_unique()

adata_combined = adata_1914_human.concatenate(adata_1914_rat, adata_1914d_human, adata_1914d_rat, adata_1919_human, adata_1919_rat, adata_1919d_human, adata_1919d_rat, batch_categories=['1914_human', '1914_rat', '1914d_human', '1914d_rat', '1919_human', '1919_rat', '1919d_human', '1919d_rat'], join='outer')

adata_human_combined = adata_1914_human.concatenate(adata_1914d_human, adata_1919_human, adata_1919d_human, batch_categories=['1914_human', '1914d_human', '1919_human', '1919d_human'], join='outer')

adata_human_combined.obs.batch.value_counts()



NameError: name 'adata_1914_human' is not defined

In [ ]:
rf_total_combined = 'final_data/total_combined.h5ad'
rf_human_combined = 'final_data/human_combined.h5ad'

running_on = [adata_combined, adata_human_combined]
saving_to = [rf_total_combined, rf_human_combined]

# We want to quickly write everything to our results files:
def write_results(running_on, saving_to):
    for i in range(len(running_on)):
        running_on[i].write(saving_to[i])
        print('Wrote to ' + saving_to[i])
        
write_results(running_on, saving_to)

## Filtering
1) Filter any cells that have less than 200 genes expressed.
2) Filter any genes that are expressed in less than 2 cells.
3) Annotating any known mitochondrial genes, as they introduce unncessary noise.
4) Filter out any cells with more than 2500 genes.
5) Filter out any cells of which more than 5% of their genes are mitochondrial.

We are also computing QC metrics that we will use throughout. These include:
1) total_genes_by_count: single number for how many genes are present in a cell.
2) n_genes_by_count: single number for the number of genes with at least 1 count in a cell.
3) total_counts: a single number for the total amount of counts expressed by every gene in a cell.
4) pct_counts_mt: The proportion of total counts for a cell which are from mitochondrial genes.

There are others, but are not important for now...


In [ ]:

# filter cells with less than 200 genes expressed
def filter_200(running_on):
    for item in running_on:
        sc.pp.filter_cells(item, min_genes=200)
    
# filter genes expressed in less than 2 cells
def filter_2(running_on):
    for item in running_on:
        sc.pp.filter_genes(item, min_cells=2)
        
# annotate the group of mitochondrial genes as 'mt'
def annotate_mt(running_on):
    for item in running_on:
        item.var['mt'] = item.var_names.str.startswith('MT-')
# compute QC metrics for all datasets
def qc_metrics(running_on):
    for item in running_on:
        sc.pp.calculate_qc_metrics(item, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)


# filter cells with more than 2500 genes
def filter_2500(running_on):
    for item in running_on:
        item = item[item.obs.n_genes_by_counts < 2500, :]

# filter cells with more than 5% mitochondrial genes
def filter_5_mt(running_on):
    for item in running_on:
        item = item[item.obs.pct_counts_mt < 5, :]

    


In [ ]:
# Aprrox. 2.5 sec per group
filter_200(running_on)

# Aprrox. 0.5 sec per group
filter_2(running_on)


# Aprrox. 0.5 sec per group
annotate_mt(running_on)

# Aprrox. 0.5 sec per group
qc_metrics(running_on)

# Aprrox. 0.5 sec per group
filter_2500(running_on)

# Aprrox. 0.5 sec per group
filter_5_mt(running_on)

In [ ]:

sns.jointplot(
        data=adata_combined.obs,
        x="total_counts",
        y="n_genes_by_counts",
        kind="hex",
    )

sns.histplot(adata_human_combined.obs["pct_counts_mt"])

sc.pl.violin(adata_combined, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
                jitter=0.4, multi_panel=True)


## Normalization
It is important to normalize our data. Since we are combining different datasets and have done some filtering, we want to make sure that no genes or cells are over represented. Therefore we going to normalize on a per cell basis by essentially dividing all gene counts by the total count data for every cell. 

We will also put them on a log basis to make visualization easier. Just note this for future visualizations.

In [ ]:
# Normalize the data
def normalize_data(running_on):
    for item in running_on:
        sc.pp.normalize_total(item, target_sum=1e4)

# Putting on a log scale
def log1p(running_on):
    for item in running_on:
        sc.pp.log1p(item)

def variablize(running_on):
    for item in running_on:
        sc.pp.highly_variable_genes(item, min_mean=0.0125, min_disp=0.5, batch_key='batch')

def save_raw(running_on):
    for item in running_on:
        item.raw = item

def filter_highly_variable(running_on):
    for item in running_on:
        item = item[:, item.var.highly_variable]

def dedensify(running_on):
    for item in running_on:
        # remove random cells
        item = item[np.random.choice(item.shape[0], 10000, replace=False), :]
        
def regress_out(running_on):
    for item in running_on:
        sc.pp.regress_out(item, ['total_counts', 'pct_counts_mt'])
        
def scale_data(running_on, max=10):
    for item in running_on:
        sc.pp.scale(item, max_value=max)

def batch_correct(running_on):
    for item in running_on:
        sc.pp.combat(item, key='batch', inplace=True)
        

In [ ]:
# Aprrox. 0.5 sec per group
normalize_data(running_on)

# Aprrox. 0.5 sec per group
log1p(running_on)

variablize(running_on)

# Aprrox. 0.1 sec per group
save_raw(running_on)

# Aprrox. 0.5 sec per group
filter_highly_variable(running_on)

dedensify(running_on)

# If you don't dedensify, this regression will take a long time
regress_out(running_on)

# Aprrox. 1 sec per group
scale_data(running_on)

batch_correct(running_on)

### Visualizations, Dimensional Reducations, and Clustering

In [ ]:
def do_pca(running_on):
    for item in running_on:
        sc.tl.pca(item, svd_solver='arpack')

def do_nearest_neighbour(running_on):
    for item in running_on:
        sc.pp.neighbors(item, n_neighbors=40, n_pcs=20)
        
def do_leiden(running_on):
    for item in running_on:
        sc.tl.leiden(item)
        
def do_louvain(running_on):
    for item in running_on:
        sc.tl.louvain(item)
        

def do_paga(running_on):
    for item in running_on:
        sc.tl.paga(item)
        sc.pl.paga(item, plot=True)
         
def do_umap(running_on):
    for item in running_on:
      sc.tl.umap(item, spread=0.5, min_dist=0.4)
    
def do_tsne(running_on):
    for item in running_on:
        sc.tl.tsne(item)
        

In [ ]:
# quick visualisation of the highly variable genes
sc.pl.highly_variable_genes(adata_combined)

# Aprrox. 40 sec per group
do_pca(running_on)

# plotting pca, we should use color...
sc.pl.pca(adata_human_combined, color='batch')

# Taking a look at how impactful each principal component is
sc.pl.pca_variance_ratio(adata_combined, log=True)

write_results(running_on, saving_to) 

# Aprrox. 10 sec per group
do_nearest_neighbour(running_on)

# Aprrox. 5 sec per group
do_leiden(running_on)

do_louvain(running_on)

# Aprrox. 0.5 sec per group
do_paga(running_on)

# Aprrox. 15 sec per group
do_umap(running_on)

use_raw = False
color = ['louvain']
sc.pl.umap(adata_human_combined, color=color, use_raw=use_raw)

# Aprrox. 1.5 minutes per group
do_tsne(running_on)

use_raw = False
color = ['louvain']

# plotting tsne
sc.pl.tsne(adata_human_combined, color=color, use_raw=use_raw)

write_results(running_on, saving_to)


In [ ]:
number_of_genes = 25
method = "t-test" # an alternative to try is wilconxon

def rank_gene_groups(running_on):
    for item in running_on:
        sc.tl.rank_genes_groups(item, 'leiden', method=method)
        sc.pl.rank_genes_groups(item, n_genes=number_of_genes, sharey=False, pts=True)

# Aprrox. 10 sec per group
rank_gene_groups(running_on)